# Biological DataBase Design with SQLite 🧬🗂

### Bio Eng C131 | Lab 4 Adaptation | Fall 2020
### Cameron Hirsh  

## Overview

#### Background

The lab I'm working in studies metabolism. Over the years, many students and post-docs have studied enzymes from organisms across the tree of life, from flies, worms, mice, and E. coli. My PI wants me to build a database of these enzymes that future researchers in the lab can use going forward. 

We will focus on three pathways: 

1. glycolysis 
2. the citric acid cycle 
3. pentose phosphate pathway.

We will consider genes in these pathways from:

1. Drosophila
2. E. coli
3. humans

Researchers want to be able to query for:
* a gene, and retreive information on all enzymes associated with the gene (in regards to the 3 metabolic pathways)
* a pathway, and retreive information on all genes associated with the pathway.

#### Approach

To do this, I will construct a database using remote data pulled from [KEGG](https://www.genome.jp/kegg-bin/show_pathway?map00010), [NCBI](https://www.ncbi.nlm.nih.gov/) and then join the information into a structure that supports these types of queries in a simple and readable fashion. I will construct this database using SQLite supported by python.

I will have a:

1. _pathway table:_ This table will detail the three metabolic pathways we are studying including the name of the pathway and what the pathway entails.


| id | Pathway Name | Decription |
| --- | --- | --- |
| primary key | name | description |

2. _enzyme table:_ data from 4 enzymes from each pathway.

| id | Enzyme Name | Function | EC # | pathway |
| --- | --- | --- | --- | --- |
| id | name | function | number | pathway |

3. _gene table:_ genes representing each of the 12 enzymes for each of 3 organisms.

| id | Gene Name | Organism | Enzyme | Enzyme ID | Sequence |
| --- | --- | --- | --- | --- | --- |
| id | name | organism | enzyme | id for enzyme | Nucleic Acids |

I will then Join these tables to allow for querrying the desired information simply and clearly.

## Setup

In [2]:
# Setup

import sqlite3
import pandas as pd
from Bio import SeqIO

conn = sqlite3.connect('metabolism.db')
c = conn.cursor()


## Step 1: Pathway Information

Using KEGG adn NCBI to create custom pathway data file, similar to data file that may be found from a remote source, I will parse the data into the pathway data table.

In [3]:
# Pathway table

# Clear previous Table data
c.execute("DELETE FROM pathway WHERE id>0;")

c.execute('''CREATE TABLE IF NOT EXISTS pathway
                 (id INT PRIMARY KEY ASC, name TEXT, description TEXT )''')

pd.read_sql_query("SELECT * FROM pathway;", conn)

,id,name,description


In [4]:
# Populate Pathway Table

# Add data using SQL Query
pathway_file = open("pathway_data.txt")

counter = 1
for i, line in enumerate(pathway_file):
    
    line = line.replace('\n', '')
    
    
    if i < 5:
        continue
    elif i % 3 == 2:
        name = line
    elif i % 3 == 0:
        description = line
    else:
        c.execute("INSERT INTO pathway (id, name, description) VALUES (" + str(counter) + ", '" + name + "', '" + description + "' )")
        counter += 1
        

# View the Table
pathway_df = pd.read_sql_query("SELECT * FROM pathway;", conn)
pathway_df = pathway_df.style.set_properties(**{'text-align': 'left'})
pathway_df


,id,name,description
0,1,Glycolysis,Linear metabolic pathway of enzyme-catalyzed reactions that convert glucose into two molecules of pyruvate in the presence of oxygen or into two molecules of lactate in the absence of oxygen.
1,2,"Citric Acid Cycle (TCA, Krebs)","Series of chemical reactions to release stored energy through the oxidation of acetyl-CoA derived from carbohydrates, fats, and proteins"
2,3,Pentose Phosphate,"Parallel to glycolysis. It generates NADPH and pentoses as well as ribose 5-phosphate, a precursor for the synthesis of nucleotides. While the pentose phosphate pathway does involve oxidation of glucose, its primary role is anabolic rather than catabolic."


looks good, lets grab the enzyme data. 

## Step 2: Enzyme Data From Pathways

Using KEGG and NCBI to create custom enzyme data file, similar to data file that may be found from a remote source, I will parse the data into the enzyme data table.

In [5]:
# Enzyme table

# Clear previous Table data
try :
    c.execute("DELETE FROM enzyme WHERE id>0;")
except:
    pass


c.execute('''CREATE TABLE IF NOT EXISTS enzyme
                (id INT PRIMARY KEY ASC, name TEXT, pathway TEXT, function TEXT, ecnum TEXT )''')

pd.read_sql_query("SELECT * FROM enzyme;", conn)


,id,name,pathway,function,ecnum


In [6]:
# Populate Enzyme Table

# Add data using SQL Query

enzyme_file = open("enzyme_data.txt")

counter = 1
for i, line in enumerate(enzyme_file):
    
    line = line.replace('\n', '')
    
    
    if i < 7:
        continue
    elif i % 5 == 2:
        name = line
    elif i % 5 == 3:
        pathway = line
    elif i % 5 == 4:
        function = line
    elif i % 5 == 0:
        ecnum = line
    else:
        c.execute("INSERT INTO enzyme (id, name, pathway, function, ecnum) VALUES (" 
                  + str(counter) + ", '" + name + "', '" + pathway + "', '" + function + "', '" + ecnum + "' )")
        counter += 1
        


# View the Table
pd.read_sql_query("SELECT * FROM enzyme;", conn)


,id,name,pathway,function,ecnum
0,1,hexokinase,glycolysis,"phosphorylates hexoses (six-carbon sugars), fo...",2.7.1.1
1,2,phosphoglucose isomerase,glycolysis,interconverts glucose-6-phosphate (G6P) and fr...,5.3.1.9
2,3,phosphofructokinase,glycolysis,phosphorylates fructose 6-phosphate.,2.7.1.11
3,4,aldolase,glycolysis,catalyzes a reversible reaction that splits th...,4.1.2.13
4,5,Citrate synthase,citric acid cycle,catalyze the conversion of acetyl-CoA [H3CC(=O...,2.3.3.1
5,6,Aconitase,citric acid cycle,enzyme that catalyses the stereo-specific isom...,4.2.1.3
6,7,Isocitrate dehydrogenase,citric acid cycle,catalyzes the oxidative decarboxylation of iso...,1.1.1.42
7,8,Fumarase,citric acid cycle,catalyzes the reversible hydration/dehydration...,4.2.1.2
8,9,G6PD,pentose phosphate,catalyzes the chemical reaction: D-glucose 6-p...,2.3.3.1
9,10,6PGD,pentose phosphate,forms ribulose 5-phosphate from 6-phosphogluco...,1.1.1.44


## Step 3: Gather & Consolidate Remote Gene Data

Pull remotely 3 Fasta Format Files of nucleic acid sequences representing most relevant search results from [NCBi database](usinghttps://www.ncbi.nlm.nih.gov/) to find genes for the selected enzymes for each of the 3 organisms. Will parse each fasta file into database.

In [17]:
# Gene table

# Clear previous Table data
try:
    c.execute("DELETE FROM gene WHERE id>0;")
except:
    pass

c.execute('''CREATE TABLE IF NOT EXISTS gene
                (id INT PRIMARY KEY ASC, name TEXT, organism TEXT, enzyme TEXT, enzyme_id INT, sequence TEXT )''')

pd.read_sql_query("SELECT * FROM gene;", conn)

,id,name,organism,enzyme,enzyme_id,sequence


In [18]:
# Enzyme Matching Functionality

enzyme_df = pd.read_sql_query("SELECT * FROM enzyme;", conn)
enzymes = enzyme_df['name']

# takes in messy enzyme name and returns enzyme id for a match
def helper(enzyme_name):
    for i in range(len(enzymes)):
        if enzymes[i].lower() in enzyme_name.lower():
            return i + 1


In [19]:
# Nested Loop to Parse each of 3 files adding FASTA content to Gene Database


# Use Seq.IO to Parse Fasta File and add genes to table


files = ['homo.fa', 'drosophila.fa', 'ecoli.fa']
organisms = ['Homo Sapien', 'Drosophila', 'E. coli']
split_by = ['sapiens ', 'Drosophila', 'coli ']

gene_id = 1

for i in range(0,3):
    
    for gene in SeqIO.parse(files[i], 'fasta'):
        
        name = gene.id
        organism = organisms[i]
        enzyme = gene.description.split(split_by[i])[1]
        enzyme_id = helper(enzyme)
        seq = str(gene.seq)
        
        c.execute("INSERT INTO gene (id, name, organism, enzyme, enzyme_id, sequence) VALUES (" 
                  + str(gene_id) + ", '" + name + "', '" + organism + "', '" + enzyme + "', '" + str(enzyme_id) + "', '" + seq + "' )")
        gene_id += 1
        
        
# Use SQL to manually update the mis-translated enzymes due to inconsistent naming syntax / missing information
c.execute("UPDATE gene SET enzyme_id = 10 WHERE id = 10;")
c.execute("UPDATE gene SET enzyme_id = 12 WHERE id = 12;")
c.execute("UPDATE gene SET enzyme_id = 12 WHERE id = 24;")
c.execute("UPDATE gene SET enzyme_id = 9 WHERE id = 33;")
c.execute("UPDATE gene SET enzyme_id = 10 WHERE id = 34;")
c.execute("UPDATE gene SET enzyme_id = 12 WHERE id = 36;")   
        
        
pd.read_sql_query("SELECT * FROM gene;", conn)

,id,name,organism,enzyme,enzyme_id,sequence
0,1,M75126.1,Homo Sapien,"hexokinase 1 (HK1) mRNA, complete cds",1,CCGCCGGAGGACCACGGCTCGCCAGGGCTGCGGAGGACCGACCGTC...
1,2,XX_0,Homo Sapien,phosphoglucose isomerase (PGI),2,N
2,3,M64784.1,Homo Sapien,"phosphofructokinase (PFK) mRNA, complete cds",3,GAATTCCGAAGTACTTGGAAGAGATCGCCACACAGATGCGCACCAG...
3,4,XX_0,Homo Sapien,aldolase,4,N
4,5,AF053631.1,Homo Sapien,"citrate synthase mRNA, complete cds",5,CCGGGCTGGGCGCCGCCGCCGGTTCGTCTACTCTTTCCTTCAGCCG...
5,6,BC014092.2,Homo Sapien,"aconitase 2, mitochondrial, mRNA (cDNA clone M...",6,CTCATCTTTGTCAGTGCACAAAATGGCGCCCTACAGCCTACTGGTG...
6,7,KU639670.1,Homo Sapien,voucher NGX277 isocitrate dehydrogenase (IDH2)...,7,TCCCAATGGAACTATCCGGAACATCCTGGGGGGGACTGTCTTCCGG...
7,8,XX_0,Homo Sapien,Fumarase,8,N
8,9,MT640143.1,Homo Sapien,"isolate ml2608 Yunnan G6PD (G6PD) gene, partia...",9,ATGGCAGAGCAGGTGGCCCTGAGCCGGACCCAGGTGTGCGGGATCC...
9,10,NM_002631.4,Homo Sapien,"phosphogluconate dehydrogenase (PGD), transcri...",10,CCCTCACTCGTCCTCCGCGCGTCGCCGCTCTTCGGTTCTGCTCTGT...


Now that we have the database in order we can use SQL JOIN functionality to create tables that can be queried for our intended querying purposes.

## Last Step: Join Tables for Desired Query Capability